<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_overlap_100_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    969      0 --:--:-- --:--:-- --:--:--   972
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_100.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,heel pain
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [18]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [19]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'i',
 'and',
 'the',
 'to',
 'my',
 'a',
 'of',
 'it',
 'in',
 'pain',
 'for',
 'have',
 'was',
 'lipitor',
 'this',
 'on',
 'that',
 'not',
 'but',
 'is',
 'with',
 'taking',
 'had',
 'after',
 '-',
 'me',
 'muscle',
 'drug',
 'side',
 'all',
 "n't",
 'am',
 'effects',
 'cholesterol',
 ')',
 'so',
 '(',
 'been',
 'as',
 'back',
 'from',
 'mg',
 'be',
 'no',
 'at',
 'years',
 'take',
 'about',
 '/',
 'has',
 '!',
 'did',
 'will',
 'now',
 'doctor',
 'very',
 'day',
 'months',
 'like',
 'or',
 'off',
 'severe',
 'do',
 'up',
 'are',
 'started',
 'if',
 'when',
 'stopped',
 'could',
 'days',
 'took',
 '3',
 'also',
 'symptoms',
 'time',
 'problems',
 'would',
 'loss',
 '2',
 'weeks',
 'some',
 'legs',
 'just',
 'can',
 'then',
 'went',
 'more',
 'one',
 'get',
 'out',
 'feel',
 'leg',
 'over',
 'any',
 'an',
 'you',
 'by',
 'he',
 'ago',
 'only',
 '10',
 'still',
 'lower',
 'cramps',
 '

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.864796,4.199234,0.214445,03:59


In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.966835,3.707342,0.294834,04:01


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.703233,3.676369,0.298661,04:04


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.666519,3.653099,0.302264,03:53


In [0]:
learn.save_encoder('AskAPatient_overlap_100_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('AskAPatient_overlap_100_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.909884,3.433010,0.422485,01:15


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.784190,2.912418,0.507692,01:28


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.280926,2.601950,0.563314,03:39


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.975915,2.318142,0.598817,03:40
1,1.722843,2.183626,0.627219,03:54


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.607382,1.971067,0.666272,03:58
1,1.405948,1.853383,0.680473,03:49


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.367181,1.757965,0.701775,04:01
1,1.183400,1.675906,0.712426,03:49


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.095126,1.648734,0.718343,03:34
1,1.012449,1.572554,0.728994,03:44


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.977703,1.589157,0.730178,04:07
1,0.890492,1.508539,0.745562,03:55


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.850255,1.500385,0.749112,03:43
1,0.850117,1.474221,0.765680,04:00
2,0.741772,1.446068,0.773965,03:40
3,0.666839,1.444188,0.772781,04:02


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.692048,1.436568,0.770414,03:53
1,0.706504,1.456292,0.773965,04:00
2,0.645730,1.399302,0.765680,03:34
3,0.609185,1.389964,0.776331,03:51


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.608033,1.444241,0.777515,03:38
1,0.638737,1.386264,0.778698,03:56
2,0.586928,1.417564,0.779882,03:46
3,0.525892,1.403488,0.782248,03:28


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.555085,1.367453,0.786982,03:42
1,0.583611,1.435159,0.777515,03:34
2,0.540544,1.407776,0.782248,03:19
3,0.490326,1.394831,0.782248,03:26


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.486331,1.439351,0.783432,03:14
1,0.543580,1.489342,0.781065,03:21
2,0.501646,1.460202,0.781065,03:26
3,0.473607,1.447644,0.783432,03:29


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.495854,1.481568,0.783432,03:42
1,0.510547,1.527076,0.782248,03:27
2,0.498895,1.505322,0.782248,03:47
3,0.431602,1.456035,0.786982,03:54


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.459017,1.509238,0.781065,03:57
1,0.490573,1.472147,0.784615,03:39
2,0.465639,1.496054,0.782248,03:56
3,0.436722,1.427544,0.790533,03:29


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.457135,1.495458,0.790533,03:50
1,0.491343,1.607300,0.789349,03:34
2,0.451498,1.488414,0.792899,03:51
3,0.446686,1.540898,0.786982,03:32


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.426298,1.520067,0.782248,03:39
1,0.470325,1.493839,0.791716,04:09
2,0.428898,1.472268,0.796450,03:44
3,0.441122,1.542107,0.794083,03:50


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.415892,1.534775,0.789349,03:20
1,0.452456,1.587534,0.794083,03:52
2,0.459239,1.562230,0.797633,03:48
3,0.411162,1.478033,0.797633,03:36


In [49]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.420443,1.538360,0.789349,03:44
1,0.464089,1.560760,0.783432,03:48
2,0.416741,1.464584,0.791716,03:53
3,0.408611,1.476775,0.792899,03:44


In [50]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.415537,1.481341,0.794083,03:40
1,0.452460,1.477406,0.792899,03:50
2,0.430792,1.450018,0.798817,03:35
3,0.417846,1.524817,0.798817,03:27


In [51]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.398858,1.462880,0.795266,03:27
1,0.440247,1.509242,0.791716,03:40
2,0.415434,1.505874,0.797633,03:23
3,0.384034,1.474962,0.800000,03:21


In [52]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.422378,1.562178,0.786982,03:35
1,0.427152,1.529907,0.789349,03:39
2,0.431824,1.554734,0.795266,03:37
3,0.385184,1.522604,0.795266,03:47


In [53]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.409169,1.547929,0.792899,03:26
1,0.430200,1.591822,0.792899,03:30
2,0.408872,1.581686,0.796450,03:59
3,0.375887,1.499774,0.796450,03:30


In [0]:
# save the best model

learn.save_encoder('AskAPatient_overlap_100')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [56]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

868
868
[2, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 165, 20, 1035, 1035, 384, 246, 27, 27, 31, 31, 32, 34, 289, 289, 289, 289, 289, 289, 289, 37, 37, 655, 44, 45, 49, 49, 49, 49, 49, 49, 49, 49, 100, 100, 100, 100, 100, 100, 100, 100, 100, 165, 53, 53, 58, 59, 60, 62, 1035, 71, 71, 491, 76, 76, 122, 614, 710, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 614, 614, 614, 614, 37, 452, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 88, 268, 27, 92, 94, 826, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 96, 97, 98, 100, 100, 100, 100, 100, 100, 103, 660, 289, 289, 289, 289, 289, 117, 384, 384, 122, 630, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 127, 131, 355, 135, 355, 473, 447, 447, 447, 146, 151, 151, 158, 165, 350, 350, 658, 289, 289, 289, 864, 864, 864, 864, 864, 214, 182, 182, 289, 182, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 18

In [57]:
correct_num = 0

for i in range(0, len(predict_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

608
0.7004608294930875
